In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from skimage import io
from skimage import filters
from skimage import draw
from scipy import signal
from scipy import fftpack

def define_filter(size, shape, filter_type, rotation, *args):
    
    f_filter = np.zeros(size, dtype=np.int)
    coordinates = []
    
    if shape == 'circle':
        
        coordinates = draw.circle(size[0] / 2, size[1] / 2, *args) #radius)
    elif shape == 'ellipse':
        
        coordinates = draw.ellipse(size[0] / 2, size[1] / 2, *args, rotation=rotation) #rad1, rad2, rotation)
    
    f_filter[coordinates] = 1
    f_filter = np.fft.ifftshift(f_filter)

    if filter_type == 'highpass':
        
        f_filter = 1 - f_filter # Invert mask.
    elif filter_type == 'lowpass':
        
        pass
    
    else:
        
        print('Filter Type not recognized')
    
    return f_filter

# Reading image and initiating kernel.
# signal = io.imread('images/v_strip_10.png')
signal = io.imread('images/lena.png')
# signal = io.imread('images/tutankamon.jpg')

if len(signal.shape) > 2:
    
    signal = signal[:, :, 0]

# Compute FFT of image.
f_signal = fftpack.fft2(signal)

# Creating Filter on Frequency Domain.
# f_filter = define_filter(signal.shape, 'circle', 'lowpass', 0.0, 50) # Circle with radius 50.
# f_filter = define_filter(signal.shape, 'circle', 'lowpass', 0.0, 10) # Circle with radius 10.
# f_filter = define_filter(signal.shape, 'ellipse', 'lowpass', 0.0, 100, 5) # Ellipse with axis equal to 100 and 5.
# f_filter = define_filter(signal.shape, 'ellipse', 'lowpass', 0.0, 100, 10) # Ellipse with axis equal to 100 and 5.
# f_filter = define_filter(signal.shape, 'circle', 'highpass', 0.0, 10) # Circle with radius 50.

f_filter = define_filter(signal.shape, 'circle', 'lowpass', 0.0, 20) & define_filter(signal.shape, 'circle', 'highpass', 0.0, 1) # Bandpass.
# f_filter = define_filter(signal.shape, 'circle', 'lowpass', 0.0, 50) & define_filter(signal.shape, 'circle', 'highpass', 0.0, 10) # Bandpass.
# f_filter = define_filter(signal.shape, 'circle', 'lowpass', 0.0, 120) & define_filter(signal.shape, 'circle', 'highpass', 0.0, 80) # Bandpass.
# f_filter = define_filter(signal.shape, 'circle', 'lowpass', 0.0, 120) & define_filter(signal.shape, 'circle', 'highpass', 0.0, 10) # Bandpass.
# f_filter = define_filter(signal.shape, 'ellipse', 'lowpass', 0, 250, 10) | define_filter(signal.shape, 'ellipse', 'lowpass', 0, 10, 250) # Bandpass.
# f_filter = define_filter(signal.shape, 'ellipse', 'lowpass', -45, 200, 20) | define_filter(signal.shape, 'ellipse', 'lowpass', 45, 200, 20) # Bandpass.
# f_filter = define_filter(signal.shape, 'circle', 'highpass', 0.0, 120) | define_filter(signal.shape, 'circle', 'lowpass', 0.0, 80) # Bandpass.

f_filter = f_filter.astype(np.float)

# Mitigating Gibbs Phenomenon.
f_filter = filters.gaussian(f_filter, sigma=5)

# Pointwise multiplication.
f_result = f_signal * f_filter
f_result[f_filter == 0] = f_result[f_filter == 0] + 0.0001 # For plotting purposes.

# Inverse FFT of frequency image.
f_conv = np.real(fftpack.ifft2(f_result))

# Plotting.
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

ax1.imshow(np.abs(np.real(np.fft.fftshift(f_signal))), cmap=plt.cm.gray, norm=colors.LogNorm(vmin=5))
ax1.set_title('freq image')
ax1.set_xticks([])
ax1.set_yticks([])
ax2.imshow(np.abs(np.fft.fftshift(f_filter)), cmap=plt.cm.gray)
ax2.set_title('freq filter')
ax2.set_xticks([])
ax2.set_yticks([])
ax3.imshow(np.abs(np.real(np.fft.fftshift(f_result))), cmap=plt.cm.gray, norm=colors.LogNorm(vmin=5))
ax3.set_title('filtered image')
ax3.set_xticks([])
ax3.set_yticks([])

plt.show()

# Plotting.
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), sharey=True)

ax1.imshow(signal, cmap=plt.cm.gray)
ax1.set_title('Original Image')
ax1.set_xticks([])
ax1.set_yticks([])
ax2.imshow(f_conv, cmap=plt.cm.gray)
ax2.set_title('Result Image')
ax2.set_xticks([])
ax2.set_yticks([])

plt.show()